In [1]:
import madxmodule
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import csv
import glob
import datetime
import collections
import time
import subprocess
import os
from scipy import optimize as opt
from scipy import constants as const
from StringIO import StringIO
from matplotlib import rc,rcParams
from matplotlib.patches import Rectangle
import itertools
import h5py as hdf
from pandas import HDFStore 
# simdata
from pandas.tools.plotting import autocorrelation_plot
from pandas.tools.plotting import lag_plot
from pandas.tools.plotting import scatter_matrix

# Creating hdf5 file to store all the data

In [2]:
# creating hierarchical structure in hdf5 format
# **********************************************
hdfoutfn = 'madx_sim_bfpp_ip5_fill4707.h5'
hdfoutfile = HDFStore(hdfoutfn)

# Left of IP5 

In [5]:
# setup initial values used in the simulation
# *******************************************

bumpamplist     = [-0.0005,0.000,0.001,0.002,0.003,0.004,0.005] 
# be careful for B1 signs needs to be opposite ->Beam4
npart           = 50000
basefilelist    = [['lhcb4-twiss-bfpp-'+ i + '.tfs'] for i in ['m0p5','0p0','1p0','2p0','3p0','4p0','5p0']] 
errorstringlist = ['']
corrlist        = np.array(['MCBCH.7L5.B2', 'MCBCH.9L5.B2', 'MCBH.13L5.B2'])

# -------------------------------------------
# twiss file list
# -------------------------------------------
basetwissfilelist = [[madxmodule.Twiss('lhcb2',
                                       basefilelist[j][i]
                                       ,targetxc=bumpamplist[j],
                                       IPcycle='IP5',
                                       targetel="MQ.11L5.B2",
                                       correctorlist=corrlist,
                                       errorseq=errorstringlist[i],
                                       twisscols=madxmodule.MADtwissColumns["LHCTwiss"],
                                       beam=[madxmodule.MADX_Beam(1,seq='LHCB1',energy=82.*6370.),
                                             madxmodule.MADX_Beam(2,seq='LHCB2',energy=82.*6370.)]) 
                      for i in range(len(basefilelist[j]))] for j in range(len(basefilelist))]

# -------------------------------------------
# initial parameters for BFPP
# -------------------------------------------
initialdictlist = [[madxmodule.get_initial(fn,madxmodule.dpPb(0,-1),location='IP5') 
                    for fn in basetwissfilelist[i]] for i in range(len(basetwissfilelist))]  

flist = [fn[0] for fn in [basetwissfilelist[i] for i in range(len(basetwissfilelist))]]

basedflist = [pd.read_csv(fn,
                skiprows=range(47),
                         delim_whitespace=True,
                         names =madxmodule.MADtwissColumns["LHCTwiss"])
              for fn in flist]

# -------------------------------------------
# Storing main beam data in hdf5
# -------------------------------------------
hdfoutfile.put('/mainbeam/beam4/bumpm0p5/bumpm0p5DF',basedflist[0],format='table')
hdfoutfile.put('/mainbeam/beam4/bumpp0p0/bumpp0p0DF',basedflist[1],format='table')
hdfoutfile.put('/mainbeam/beam4/bumpp1p0/bumpp1p0DF',basedflist[2],format='table')
hdfoutfile.put('/mainbeam/beam4/bumpp2p0/bumpp2p0DF',basedflist[3],format='table')
hdfoutfile.put('/mainbeam/beam4/bumpp3p0/bumpp3p0DF',basedflist[4],format='table')
hdfoutfile.put('/mainbeam/beam4/bumpp4p0/bumpp4p0DF',basedflist[5],format='table')
hdfoutfile.put('/mainbeam/beam4/bumpp5p0/bumpp5p0DF',basedflist[6],format='table')

# -------------------------------------------
# Tracking BFPP beam
# -------------------------------------------
fileextlist = [['noerr' + i] for i in ['m0p5','0p0','1p0','2p0','3p0','4p0','5p0'] ]

transfermatrixfilelist = [[madxmodule.TransferMatrix(
                            'LHCB2',
                            'IP5',
                            'S.DS.L5.B2',
                            initialdictlist[j][i],
                            bumpamplist[j],
                            "MQ.11L5.B2",
                            corrlist,
                            IPcycle  = 'IP5',
                            errorseq = errorstringlist[i],
                            fileext  = fileextlist[j][i],
                            beam     = [madxmodule.MADX_Beam(1,seq='LHCB1',energy=82.*6370.),
                                        madxmodule.MADX_Beam(2,seq='LHCB2',energy=82.*6370.)]
                             ) 
                           for i in range(len(basetwissfilelist[j]))] for j in range(len(basefilelist))]

# --------------------------------------
# writing transfermatrixfilelist to file 
# --------------------------------------
dftransfermatrixfilelist       = [pd.DataFrame(pd.Series(data=transfermatrixfilelist[j]),
                                               columns=['filename']) 
                                  for j in range(len(transfermatrixfilelist))]

for j in range(len(dftransfermatrixfilelist)):
    dftransfermatrixfilelist[j].to_csv('transfermatrixfilenamelist'+ str(j) +'.csv',index=False)

# --------------------------------------
# writing BFPP beams to hdf5 
# --------------------------------------
dftransfermatrixfilelist = [pd.read_csv('transfermatrixfilenamelist'+ str(j) +'.csv') for j in range(7) ]

bfppfnlist = [fn for fn in [dftransfermatrixfilelist[i]['filename'].values[0] for i in range(len(basefilelist))]]
BFPPdflist   = [pd.read_csv(fn,skiprows=range(47),
                            delim_whitespace=True,
                            names =madxmodule.MADtwissColumns["RMatrixExtended"])
                for fn in bfppfnlist]

hdfoutfile.put('/bfppbeam/beam4/bumpm0p5/bumpm0p5DF',BFPPdflist[0],format='table')
hdfoutfile.put('/bfppbeam/beam4/bumpp0p0/bumpp0p0DF',BFPPdflist[1],format='table')
hdfoutfile.put('/bfppbeam/beam4/bumpp1p0/bumpp1p0DF',BFPPdflist[2],format='table')
hdfoutfile.put('/bfppbeam/beam4/bumpp2p0/bumpp2p0DF',BFPPdflist[3],format='table')
hdfoutfile.put('/bfppbeam/beam4/bumpp3p0/bumpp3p0DF',BFPPdflist[4],format='table')
hdfoutfile.put('/bfppbeam/beam4/bumpp4p0/bumpp4p0DF',BFPPdflist[5],format='table')
hdfoutfile.put('/bfppbeam/beam4/bumpp5p0/bumpp5p0DF',BFPPdflist[6],format='table')

# ---------------------------------------
# reading simdata for calculating impacts 
# ---------------------------------------
opt         = madxmodule.io.tfsDict(basetwissfilelist[0][0])
prevelinex  = opt[0]["name"].index("MB.B11L5.B2")-1
s0          = opt[0]['s'][prevelinex]

# ------------------------------------------------------------
# generating the filenames for storing the impactdistributions
# ------------------------------------------------------------
impactfilelist    = [['impactIP5left_bp22_' + ext for ext in fileextlist[j]] for j in range(len(fileextlist))]
impactfilelistext = [[fn+'.csv' for fn in impactfilelist[j]] for j in range(len(impactfilelist))]

dfimpactfilelist  = [pd.DataFrame(pd.Series(data=impactfilelistext[j]),columns=['filename']) 
                     for j in range(len(impactfilelist))]


for j in range(len(dfimpactfilelist)):
    dfimpactfilelist[j].to_csv('impactfilenamelist' + str(j) +'.csv',index=False)
    
# --------------------------------------
# generating the impact distributions
# --------------------------------------
for j in range(len(transfermatrixfilelist)):
    for i in range(len(transfermatrixfilelist[j])):
        madxmodule.impactcoordinates6D(madxmodule.TrackSigmaMatrix(transfermatrixfilelist[j][i],
                                                                   'MB.B11L5.B2',initialdictlist[j][i],
                                                                   npart),
                                       0.02315,
                                       madxmodule.lhcradius,
                                       s0,
                                       madxmodule.dipolelength,
                                       madxmodule.dpPb(0,-1),
                                       madxmodule.ionmass,
                                       -madxmodule.electronmassgev,
                                       madxmodule.get_p(transfermatrixfilelist[j][i]),
                                       impactfilelist[j][i],
                                       beam4=True)

# ----------------------------------------
# writing the impact distributions to hdf5
# ----------------------------------------

hdfoutfile.put('/bfppbeam/beam4/bumpm0p5/bumpm0p5DFimpact',pd.read_csv(impactfilelistext[0][0]),format='table')
hdfoutfile.put('/bfppbeam/beam4/bumpp0p0/bumpp0p0DFimpact',pd.read_csv(impactfilelistext[1][0]),format='table')
hdfoutfile.put('/bfppbeam/beam4/bumpp1p0/bumpp1p0DFimpact',pd.read_csv(impactfilelistext[2][0]),format='table')
hdfoutfile.put('/bfppbeam/beam4/bumpp2p0/bumpp2p0DFimpact',pd.read_csv(impactfilelistext[3][0]),format='table')
hdfoutfile.put('/bfppbeam/beam4/bumpp3p0/bumpp3p0DFimpact',pd.read_csv(impactfilelistext[4][0]),format='table')
hdfoutfile.put('/bfppbeam/beam4/bumpp4p0/bumpp4p0DFimpact',pd.read_csv(impactfilelistext[5][0]),format='table')
hdfoutfile.put('/bfppbeam/beam4/bumpp5p0/bumpp5p0DFimpact',pd.read_csv(impactfilelistext[6][0]),format='table')   

MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2


# Right of IP5

In [7]:
# setup initial values used in the simulation
# *******************************************

bumpamplist     = [0.0005,0.000,-0.001,-0.002,-0.003,-0.004,-0.005] 
# be careful for B1 signs needs to be opposite ->Beam4
npart           = 50000
basefilelistb1  =[['lhcb1-twiss-bfpp-'+ i + '.tfs'] for i in ['m0p5','0p0','1p0','2p0','3p0','4p0','5p0']] 
errorstringlist = ['']
corrlistb1      = np.array(['MCBCH.7R5.B1', 'MCBCH.9R5.B1', 'MCBH.13R5.B1'])

# -------------------------------------------
# twiss file list
# -------------------------------------------
basetwissfilelistb1 = [[madxmodule.Twiss('lhcb1',
                                         basefilelistb1[j][i],
                                         targetxc = bumpamplist[j],
                                         IPcycle  = 'IP5',
                                         targetel = "MQ.11R5.B1",
                                         correctorlist = corrlistb1,
                                         fileloading = '''
system,"ln -fns  /afs/cern.ch/eng/lhc/optics/runII/2015/ db5";
call, file="db5/lhc_as-built.seq";
call, file="db5/opt_inj_colltunes.madx";
call, file="db5/opt_800_800_800_3000_ion_coll.madx";''',
                                         errorseq  = errorstringlist[i],
                                         twisscols = madxmodule.MADtwissColumns["LHCTwiss"],
                                         beam=[madxmodule.MADX_Beam(1,seq='LHCB1',energy=82.*6370.),
                                               madxmodule.MADX_Beam(2,seq='LHCB2',energy=82.*6370.)]) 
                      for i in range(len(basefilelistb1[j]))] for j in range(len(basefilelistb1))]




# -------------------------------------------
# initial parameters for BFPP
# -------------------------------------------
initialdictlistb1 = [[madxmodule.get_initial(fn,
                                             madxmodule.dpPb(0,-1),
                                             location='IP5'
                                            ) 
                      for fn in basetwissfilelistb1[i]] for i in range(len(basetwissfilelistb1))]  

flistb1 = [fn[0] for fn in [basetwissfilelistb1[i] for i in range(len(basetwissfilelistb1))]]

basedflistb1 = [pd.read_csv(fn,
                            skiprows=range(47),
                            delim_whitespace=True,
                            names =madxmodule.MADtwissColumns["LHCTwiss"])
                for fn in flistb1]

# -------------------------------------------
# Storing main beam data in hdf5
# -------------------------------------------
hdfoutfile.put('/mainbeam/beam1/bumpm0p5/bumpm0p5DF',basedflistb1[0],format='table')
hdfoutfile.put('/mainbeam/beam1/bumpp0p0/bumpp0p0DF',basedflistb1[1],format='table')
hdfoutfile.put('/mainbeam/beam1/bumpp1p0/bumpp1p0DF',basedflistb1[2],format='table')
hdfoutfile.put('/mainbeam/beam1/bumpp2p0/bumpp2p0DF',basedflistb1[3],format='table')
hdfoutfile.put('/mainbeam/beam1/bumpp3p0/bumpp3p0DF',basedflistb1[4],format='table')
hdfoutfile.put('/mainbeam/beam1/bumpp4p0/bumpp4p0DF',basedflistb1[5],format='table')
hdfoutfile.put('/mainbeam/beam1/bumpp5p0/bumpp5p0DF',basedflistb1[6],format='table')

# -------------------------------------------
# Tracking BFPP beam
# -------------------------------------------
fileextlistb1 = [['b1noerr' + i] for i in ['m0p5','0p0','1p0','2p0','3p0','4p0','5p0'] ]

transfermatrixfilelistb1 = [[madxmodule.TransferMatrix('LHCB1','IP5','E.DS.R5.B1',
                                                       initialdictlistb1[j][i],
                                                       bumpamplist[j],
                                                       "MQ.11R5.B1",
                                                       corrlistb1,
                                                       IPcycle='IP5',
                                                       errorseq = errorstringlist[i],
                                                       fileloading = '''
system,"ln -fns  /afs/cern.ch/eng/lhc/optics/runII/2015/ db5";
call, file="db5/lhc_as-built.seq";
call, file="db5/opt_inj_colltunes.madx";
call, file="db5/opt_800_800_800_3000_ion_coll.madx";''',
                                                       fileext = fileextlistb1[j][i],
                                                       beam =[madxmodule.MADX_Beam(1,seq='LHCB1',energy=82.*6370.),
                                                              madxmodule.MADX_Beam(2,seq='LHCB2',energy=82.*6370.)]
                             ) for i in range(len(basefilelistb1[j]))] for j in range(len(basefilelistb1))]

# --------------------------------------
# writing transfermatrixfilelist to file 
# --------------------------------------
dftransfermatrixfilelistb1       = [pd.DataFrame(pd.Series(data=transfermatrixfilelistb1[j]),
                                                 columns=['filename']) 
                                    for j in range(len(transfermatrixfilelistb1))]

for j in range(len(dftransfermatrixfilelistb1)):
    dftransfermatrixfilelistb1[j].to_csv('transfermatrixfilenamelistb1'+ str(j) +'.csv',index=False)

# --------------------------------------
# writing BFPP beams to hdf5 
# --------------------------------------

dftransfermatrixfilelistb1 = [pd.read_csv('transfermatrixfilenamelistb1'+ str(j) +'.csv') for j in range(7) ]

bfppfnlistb1 = [fn for fn in [dftransfermatrixfilelistb1[i]['filename'].values[0] 
                              for i in range(len(basefilelistb1))]]

BFPPdflistb1   = [pd.read_csv(fn,skiprows=range(47),
                            delim_whitespace=True,
                            names =madxmodule.MADtwissColumns["RMatrixExtended"])
                for fn in bfppfnlistb1]

hdfoutfile.put('/bfppbeam/beam1/bumpm0p5/bumpm0p5DF',BFPPdflistb1[0],format='table')
hdfoutfile.put('/bfppbeam/beam1/bumpp0p0/bumpp0p0DF',BFPPdflistb1[1],format='table')
hdfoutfile.put('/bfppbeam/beam1/bumpp1p0/bumpp1p0DF',BFPPdflistb1[2],format='table')
hdfoutfile.put('/bfppbeam/beam1/bumpp2p0/bumpp2p0DF',BFPPdflistb1[3],format='table')
hdfoutfile.put('/bfppbeam/beam1/bumpp3p0/bumpp3p0DF',BFPPdflistb1[4],format='table')
hdfoutfile.put('/bfppbeam/beam1/bumpp4p0/bumpp4p0DF',BFPPdflistb1[5],format='table')
hdfoutfile.put('/bfppbeam/beam1/bumpp5p0/bumpp5p0DF',BFPPdflistb1[6],format='table')


# ---------------------------------------
# reading simdata for calculating impacts 
# ---------------------------------------
optb1         = madxmodule.io.tfsDict(basefilelistb1[0][0])
prevelinexb1  = optb1[0]["name"].index("MB.B11R5.B1")-1
s0            = optb1[0]['s'][prevelinexb1]

# ------------------------------------------------------------
# generating the filenames for storing the impactdistributions
# ------------------------------------------------------------
impactfilelistb1    = [['impactIP5right_bp22_' + ext for ext in fileextlistb1[j]] 
                       for j in range(len(fileextlistb1))]
impactfilelistextb1 = [[fn+'.csv' for fn in impactfilelistb1[j]] for j in range(len(impactfilelistb1))]


dfimpactfilelistb1  = [pd.DataFrame(pd.Series(data=impactfilelistextb1[j]),columns=['filename']) 
                     for j in range(len(impactfilelistb1))]


for j in range(len(dfimpactfilelistb1)):
    dfimpactfilelistb1[j].to_csv('impactfilenamelistb1' + str(j) +'.csv',index=False)
    
# --------------------------------------
# generating the impact distributions
# --------------------------------------
for j in range(len(dftransfermatrixfilelistb1)):
    for i in range(len(dftransfermatrixfilelistb1[j])):
        madxmodule.impactcoordinates6D(madxmodule.TrackSigmaMatrix(transfermatrixfilelistb1[j][i],
                                                                   'MB.B11R5.B1',
                                                                   initialdictlistb1[j][i],
                                                                   npart),
                                       0.02315,
                                       madxmodule.lhcradius,
                                       s0,
                                       madxmodule.dipolelength,
                                       madxmodule.dpPb(0,-1),
                                       madxmodule.ionmass,
                                       -madxmodule.electronmassgev,
                                       madxmodule.get_p(transfermatrixfilelistb1[j][i]),
                                       impactfilelistb1[j][i],
                                       beam4=False)

# ----------------------------------------
# writing the impact distributions to hdf5
# ----------------------------------------
hdfoutfile.put('/bfppbeam/beam1/bumpm0p5/bumpm0p5DFimpact',pd.read_csv(impactfilelistextb1[0][0]),format='table')
hdfoutfile.put('/bfppbeam/beam1/bumpp0p0/bumpp0p0DFimpact',pd.read_csv(impactfilelistextb1[1][0]),format='table')
hdfoutfile.put('/bfppbeam/beam1/bumpp1p0/bumpp1p0DFimpact',pd.read_csv(impactfilelistextb1[2][0]),format='table')
hdfoutfile.put('/bfppbeam/beam1/bumpp2p0/bumpp2p0DFimpact',pd.read_csv(impactfilelistextb1[3][0]),format='table')
hdfoutfile.put('/bfppbeam/beam1/bumpp3p0/bumpp3p0DFimpact',pd.read_csv(impactfilelistextb1[4][0]),format='table')
hdfoutfile.put('/bfppbeam/beam1/bumpp4p0/bumpp4p0DFimpact',pd.read_csv(impactfilelistextb1[5][0]),format='table')
hdfoutfile.put('/bfppbeam/beam1/bumpp5p0/bumpp5p0DFimpact',pd.read_csv(impactfilelistextb1[6][0]),format='table')        

MCS.A11R5.B1
DRIFT_63
MB.B11R5.B1
MCS.A11R5.B1
DRIFT_63
MB.B11R5.B1
MCS.A11R5.B1
DRIFT_63
MB.B11R5.B1
MCS.A11R5.B1
DRIFT_63
MB.B11R5.B1
MCS.A11R5.B1
DRIFT_63
MB.B11R5.B1
MCS.A11R5.B1
DRIFT_63
MB.B11R5.B1
MCS.A11R5.B1
DRIFT_63
MB.B11R5.B1


In [26]:
bbeamparms = pd.read_csv(basetwissfilelistb1[0][0],nrows=45,delim_whitespace=True,
                         names=['n0','NAME','n1','twiss'])
bbeamparms =bbeamparms[['NAME','twiss']]
hdfoutfile.put('beamparam',bbeamparms,format='table')

In [27]:
print hdfoutfile

<class 'pandas.io.pytables.HDFStore'>
File path: madx_sim_bfpp_ip5_fill4707.h5
/beamparam                                           frame_table  (typ->appendable,nrows->45,ncols->2,indexers->[index])    
/bfppbeam/beam1/bumpm0p5/bumpm0p5DF                  frame_table  (typ->appendable,nrows->280,ncols->64,indexers->[index])  
/bfppbeam/beam1/bumpm0p5/bumpm0p5DFimpact            frame_table  (typ->appendable,nrows->50000,ncols->6,indexers->[index]) 
/bfppbeam/beam1/bumpp0p0/bumpp0p0DF                  frame_table  (typ->appendable,nrows->280,ncols->64,indexers->[index])  
/bfppbeam/beam1/bumpp0p0/bumpp0p0DFimpact            frame_table  (typ->appendable,nrows->50000,ncols->6,indexers->[index]) 
/bfppbeam/beam1/bumpp1p0/bumpp1p0DF                  frame_table  (typ->appendable,nrows->280,ncols->64,indexers->[index])  
/bfppbeam/beam1/bumpp1p0/bumpp1p0DFimpact            frame_table  (typ->appendable,nrows->50000,ncols->6,indexers->[index]) 
/bfppbeam/beam1/bumpp2p0/bumpp2p0DF           

In [28]:
hdfoutfile.close()

In [50]:
#removing temp files
import itertools
list1 = list(itertools.chain(*basetwissfilelist))
list2 = list(itertools.chain(*basetwissfilelistb1))
list3 = list(itertools.chain(*transfermatrixfilelist))
list4 = list(itertools.chain(*transfermatrixfilelistb1))
list5 = glob.glob('transfermatrixfilenamelist*.csv')
list6 = list(itertools.chain(*impactfilelistext))
list7 = list(itertools.chain(*impactfilelistextb1))
list8 = glob.glob('impactfile*.csv')
totlist = list1 + list2 + list3 + list4 + list5 + list6 + list7 +list8

import os
for fn in list8:
    os.remove(fn)